In [3]:
from langgraph.graph import StateGraph, END

# to construct agent state
from typing import TypedDict, Annotated
import operator

# to construct human, ai and system messages
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage

# llm 
from langchain_ollama import ChatOllama

# tool
from langchain_community.tools.tavily_search import TavilySearchResults

In [4]:
# old
import os
TAVILY_API_KEY = "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"
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

In [5]:
# old 
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [6]:
# change
from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver.from_conn_string(":memory:")

In [7]:
# old (modified: checkpointer)
class Agent:

    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,######### added
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer) ### added checkpointer
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      
                print("\n ....bad tool name....")
                result = "bad tool name, retry"
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [10]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOllama(model="llama3.1")

tool = TavilySearchResults(max_results=2)

abot = Agent(model, [tool], system=prompt, checkpointer=memory)

# Streaming Messages

In [11]:
messages = [HumanMessage(content="What is the weather in sf?")]
thread = {"configurable": {"thread_id": "1"}}

In [13]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-12T11:41:52.744613Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'tavily_search_results_json', 'arguments': {'query': 'weather in sf'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 57778046500, 'load_duration': 42799964600, 'prompt_eval_count': 256, 'prompt_eval_duration': 2839670000, 'eval_count': 49, 'eval_duration': 11968023000}, id='run-fc1805d4-7167-43ad-9fb9-7e2ace1edbf4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': '085fbc8f-4d5c-43df-94cf-d5a582c78a41', 'type': 'tool_call'}], usage_metadata={'input_tokens': 256, 'output_tokens': 49, 'total_tokens': 305})]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': '085fbc8f-4d5c-43df-94cf-d5a582c78a41', 'type': 'tool_call'}
Back to the model!
[ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \

In [14]:
messages = [HumanMessage(content="What about in la?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-12T11:48:03.2740895Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'tavily_search_results_json', 'arguments': {'query': 'weather in la'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 35163036300, 'load_duration': 17272490400, 'prompt_eval_count': 936, 'prompt_eval_duration': 10580671000, 'eval_count': 23, 'eval_duration': 7114624000}, id='run-02a48ee9-c74b-43a1-b1cc-4fb33d1e47e5-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in la'}, 'id': '93c2ae78-3d86-47f8-b50b-01018f95c4c7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 936, 'output_tokens': 23, 'total_tokens': 959})]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in la'}, 'id': '93c2ae78-3d86-47f8-b50b-01018f95c4c7', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'http

In [15]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-12T11:49:10.8453978Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'tavily_search_results_json', 'arguments': {'query': 'which city is warmer la or sf'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 11043675100, 'load_duration': 33254700, 'prompt_eval_count': 1535, 'prompt_eval_duration': 2590220000, 'eval_count': 27, 'eval_duration': 8366231000}, id='run-79617d14-7716-4978-ac53-ca465c1e44cd-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'which city is warmer la or sf'}, 'id': '8c7619c7-ea86-4a5f-aec2-13bd596ba532', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1535, 'output_tokens': 27, 'total_tokens': 1562})]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'which city is warmer la or sf'}, 'id': '8c7619c7-ea86-4a5f-aec2-13bd596ba532', 'type': 'tool_call'}
Back to the model!
{'mes